In [13]:
# allows to import own functions
import sys, os, re
root_project = re.findall(r'(^\S*TFM-master)', os.getcwd())[0]
sys.path.append(root_project)

from src.utils.help_func import results_searchcv, make_train_val_test, errors_distribution
from src.features.add_features import features_graph, features_pop

In [14]:
import pandas as pd

In [27]:
foo = pd.read_csv(f'{root_project}/data/processed/simulation_results.csv')
foo

,i_country,R0,Tc,Tr,omega,inf_pow_1,inf_pow_2,mort_pow_1,mort_pow_2,mort_pow_3,limit_deaths,n_closed,react_time,total_infected,total_death,total_removed
0,LBN,9.490802,2.214174,21.014286,0.069029,2.918571e+03,1.140780e+02,0.007146,0.005701,0.048225,72,6,23,7.509688e+09,5.191675e+08,7.521476e+09
1,KIR,3.161672,6.111805,19.323523,0.012080,1.265010e+05,2.916422e+02,0.003715,0.002368,0.007072,872,2,2,7.166248e+09,8.694994e+07,7.201020e+09
2,MOZ,16.439975,1.263448,20.771054,0.046275,8.045407e+04,2.297547e+03,0.002847,0.001771,0.026642,192,0,22,7.530737e+09,3.532065e+08,7.633573e+09
3,ERI,2.141326,1.149404,2.461249,0.014779,5.593643e+03,1.969175e+02,0.006883,0.009034,0.010026,511,9,28,6.401541e+09,9.461014e+07,6.402090e+09
4,VCT,9.327237,1.192357,11.121400,0.058201,9.880857e+03,2.862489e+02,0.013316,0.013809,0.044225,958,18,20,7.477989e+09,4.352523e+08,7.478614e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146879,KAZ,11.763182,1.447605,17.028439,0.278547,6.074571e+03,9.460448e+02,0.023576,0.015816,0.169786,76,15,29,7.504537e+09,2.090856e+09,7.506367e+09
146880,ITA,10.734467,0.201486,2.162848,0.029374,2.984051e+08,3.300158e+06,0.009123,0.012621,0.014495,434,12,21,7.512071e+09,2.206617e+08,7.512265e+09
146881,ARM,19.957722,0.655780,13.087873,0.078398,2.532314e+05,6.713776e+03,0.016655,0.016688,0.061060,788,3,14,7.535460e+09,5.919213e+08,7.550623e+09
146882,AZE,13.648353,0.278279,3.798044,0.093562,2.234922e+07,1.928435e+06,0.010039,0.004640,0.029322,314,0,12,7.552275e+09,7.113103e+08,7.602798e+09


In [30]:
foo['R0'].nunique()

100000

In [17]:
pd.read_csv(f'{root_project}/data/processed/simulation_results_v2.csv')

,i_country,R0,Tc,Tr,omega,inf_pow_1,inf_pow_2,mort_pow_1,mort_pow_2,mort_pow_3,limit_deaths,n_closed,react_time,total_infected,total_death,total_removed
0,LBN,8.741722,2.186395,19.112858,0.103831,2.148071e+03,115.327160,0.011638,0.009367,0.072329,72,20,19,3.839636e+09,3.987754e+08,3.840803e+09
1,MDV,3.799548,2.702026,10.266480,0.593460,4.885714e+01,29.500000,0.147661,0.155624,0.435345,100,7,22,7.336358e+09,4.357559e+09,7.342655e+09
2,WSM,3.015408,4.973117,14.995978,0.122095,6.971429e+01,4.824675,0.034836,0.045760,0.091644,806,1,21,5.813581e+09,7.141080e+08,5.849089e+09
3,IRN,13.114667,0.992001,13.009757,0.029942,5.323387e+05,9066.625987,0.002355,0.001390,0.015745,561,9,28,7.514257e+09,2.250144e+08,7.515610e+09
4,VCT,8.594513,1.187881,10.209260,0.276758,5.147786e+03,877.611765,0.045484,0.043943,0.184023,958,18,20,7.478609e+09,2.069940e+09,7.479284e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33106,FJI,4.037852,2.899005,11.705752,0.818785,3.607143e+01,24.350000,0.194059,0.201232,0.612500,672,8,8,3.244902e+08,2.656877e+08,3.244902e+08
33107,GHA,10.165493,0.578893,5.884732,0.273740,1.259773e+06,217642.229358,0.033650,0.025017,0.123727,7,9,6,7.932163e+07,2.171340e+07,7.932154e+07
33108,ARM,7.635228,0.703716,5.373031,0.324940,1.132873e+05,35365.200000,0.053097,0.043295,0.166360,29,6,5,5.637220e+09,1.832578e+09,5.639777e+09
33109,SGP,14.256038,1.399866,19.956545,0.255402,9.270571e+03,1228.814815,0.017837,0.011629,0.152132,741,18,28,7.514588e+09,1.919484e+09,7.515628e+09


In [18]:
pd.read_csv(f'{root_project}/data/processed/simulation_results_v3.csv')

,i_country,R0,Tc,Tr,omega,inf_pow_1,inf_pow_2,mort_pow_1,mort_pow_2,mort_pow_3,limit_deaths,n_closed,react_time,total_infected,total_death,total_removed
0,LBN,8.741722,2.186395,19.112858,0.103831,2.148071e+03,115.327160,0.011638,0.009367,0.072329,72,20,19,3.895705e+09,4.045879e+08,3.896788e+09
1,MDV,3.799548,2.702026,10.266480,0.593460,4.885714e+01,29.500000,0.147661,0.155624,0.435345,100,7,22,7.336264e+09,4.357475e+09,7.342513e+09
2,WSM,3.015408,4.973117,14.995978,0.122095,6.971429e+01,4.824675,0.034836,0.045760,0.091644,806,1,21,5.813582e+09,7.141082e+08,5.849091e+09
3,IRN,13.114667,0.992001,13.009757,0.029942,5.323387e+05,9066.625987,0.002355,0.001390,0.015745,561,9,28,7.514288e+09,2.250205e+08,7.515821e+09
4,VCT,8.594513,1.187881,10.209260,0.276758,5.147786e+03,877.611765,0.045484,0.043943,0.184023,958,18,20,7.478660e+09,2.069878e+09,7.479057e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,DEU,3.785275,4.754930,17.998719,0.292764,4.050000e+01,6.581081,0.065256,0.075975,0.211429,423,18,1,5.351671e+08,1.566770e+08,5.351670e+08
49996,MHL,16.557036,0.732844,12.133717,0.957273,1.621672e+06,414061.131148,0.087902,0.079013,0.525821,953,2,14,7.529798e+09,7.285373e+09,7.610553e+09
49997,HKG,5.901646,1.641050,9.684893,0.100018,3.583714e+03,306.239024,0.015367,0.012281,0.060047,293,9,20,7.493834e+09,7.499187e+08,7.498033e+09
49998,MMR,15.645535,0.928264,14.523181,0.903293,2.151143e+04,17638.718750,0.057810,0.030845,0.449975,390,6,18,7.508318e+09,6.793617e+09,7.520947e+09


In [31]:
foo = pd.read_csv(f'{root_project}/data/processed/simulation_results_v4_errors.csv')
foo

,i_country,R0,Tc,Tr,omega,inf_pow_1,inf_pow_2,mort_pow_1,mort_pow_2,mort_pow_3,limit_deaths,n_closed,react_time,total_infected,total_death,total_removed
0,LBN,8.741722,3.101546,27.112858,0.164458,513.642857,32.633466,0.017939,0.015749,0.121813,72,20,3,8.432086e+07,1.386692e+07,8.432068e+07
1,MDV,3.799548,4.807540,18.266480,0.710992,15.785714,6.500000,0.162896,0.184615,0.514286,100,7,6,6.070480e+05,4.315570e+05,6.070280e+05
2,WSM,3.015408,7.626157,22.995978,0.190007,30.571429,2.248227,0.049065,0.066246,0.132911,294,1,5,5.817190e+09,1.111880e+09,5.851963e+09
3,IRN,13.114667,1.602005,21.009757,0.056199,26462.571429,582.820346,0.004273,0.002940,0.035504,49,9,3,1.557424e+08,8.752254e+06,1.557422e+08
4,MMR,8.884316,3.117646,27.698156,0.683504,111.000000,32.345455,0.072716,0.063519,0.500000,485,18,9,1.670583e+09,1.141850e+09,1.670582e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,SGP,18.473903,1.031939,19.063932,0.867346,17242.500000,8752.775510,0.047399,0.026678,0.453940,183,9,7,7.503800e+09,6.508834e+09,7.504311e+09
49996,CIV,15.657956,0.799619,12.520400,0.965765,85400.000000,49171.770833,0.062628,0.031725,0.448804,63,15,4,2.503739e+07,2.418014e+07,2.503729e+07
49997,ARE,3.729777,3.696492,13.787093,0.619548,28.642857,11.933333,0.152120,0.170391,0.465649,110,3,5,1.121218e+07,6.946411e+06,1.121216e+07
49998,LAO,7.372083,1.849250,13.632825,0.701566,378.928571,217.193548,0.094251,0.074261,0.473037,496,11,8,6.538427e+09,4.590038e+09,6.542575e+09


In [32]:
foo['R0'].nunique()

50000